In [1]:

# Setup Environment

import os, warnings
warnings.filterwarnings('ignore')

# Project directories
os.makedirs('medical_rag_bot/docs', exist_ok=True)
os.chdir('/content/medical_rag_bot')

# Install required packages
!pip install -q google-generativeai langchain langchain-google-genai chromadb PyMuPDF langchain-community

print("✅ Environment ready! Docs folder: medical_rag_bot/docs")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.8 MB/s eta 0:00:

In [4]:
# API Configuration & Imports
import os
import google.generativeai as genai
from langchain.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

# Configure Gemini API
try:
    API_KEY = userdata.get('MY_GEMINI_KEY')
    os.environ["GOOGLE_API_KEY"] = API_KEY
    genai.configure(api_key=API_KEY)
    print("✅ Gemini API configured!")
except Exception as e:
    print("❌ API Key Error:", e)


✅ Gemini API configured!


In [5]:
# Document Upload
from google.colab import files
import shutil

print("📋 Upload your medical PDFs")

uploaded = files.upload()
for filename in uploaded.keys():
    if filename.endswith('.pdf'):
        shutil.move(filename, f'docs/{filename}')
        print(f"✅ Uploaded: {filename}")


📋 Upload your medical PDFs


Saving Medical_book.pdf to Medical_book.pdf
✅ Uploaded: Medical_book.pdf


In [6]:
#  Document Processing
print("📖 Processing documents...")

documents = []
for filename in os.listdir("docs"):
    if filename.endswith(".pdf"):
        loader = PyMuPDFLoader(f"docs/{filename}")
        docs = loader.load()
        documents.extend(docs)
        print(f"📄 {filename}: {len(docs)} pages")

if documents:
    splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=200)
    chunks = splitter.split_documents(documents)
    print(f"✅ Split into {len(chunks)} chunks")
else:
    print("❌ No documents found!")


📖 Processing documents...
📄 Medical_book.pdf: 637 pages
✅ Split into 2784 chunks


In [14]:
# Vector Database Setup
print("🧬 Creating Chroma Vector DB...")

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectordb = Chroma.from_documents(chunks, embeddings, persist_directory="medical_db")
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)

print("✅ Vector DB + LLM ready!")



🧬 Creating Chroma Vector DB...
✅ Vector DB + LLM ready!


In [16]:
#  Hybrid Q&A Functions
# caching
import hashlib
cache = {}  # Simple memory cache

def cached_hybrid_query(query: str):
    """Cache responses to avoid repeated API calls"""
    # Create a simple key from the query
    cache_key = query.lower().strip()

    # Check if we've seen this question before
    if cache_key in cache:
        print("💡 Using cached response!")
        return cache[cache_key]

    # If new question, process normally
    print("🔄 Processing new query...")
    result = hybrid_query(query)

    # Save result for next time
    cache[cache_key] = result
    return result

# Create RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",           # "map_reduce" for large docs
    return_source_documents=True  # fetch source docs for reference
)

def ask_llm_first(query: str):
    """Try LLM directly. If unsure, return None."""
    system_prompt = f"""
    You are a medical AI assistant. Answer only if you are confident.
    If you don’t know, reply exactly: "I don't know."

    Question: {query}
    """
    response = llm.invoke(system_prompt)
    content = response.content if hasattr(response, "content") else str(response)

    if "i don't know" in content.lower():
        return None
    return content

def ask_with_rag(query: str):
    """Fallback to RetrievalQA if LLM is unsure."""
    result = qa_chain.invoke({"query": query})
    answer = result["result"]

    sources = [doc.metadata.get("source", "Unknown") for doc in result["source_documents"]]
    sources = list(dict.fromkeys(sources))  # remove duplicates
    return answer, sources

def hybrid_query(query: str):
    """Main hybrid function: LLM first → fallback to RAG."""
    llm_answer = ask_llm_first(query)
    if llm_answer:
        return f"🧠 DIRECT: {llm_answer}"
    else:
        rag_answer, sources = ask_with_rag(query)
        return f"📚 RAG: {rag_answer}\n\n📎 Sources: {sources}"


In [ ]:
# Interactive Chat

def start_chat():
    print("🏥 Medical Chatbot (Type 'exit' to quit)\n")
    while True:
        user_q = input("💬 You: ").strip()
        if user_q.lower() == "exit":
            print("👋 Goodbye!")
            break
        answer = cached_hybrid_query(user_q)  # NEW FUNCTION
        print(f"\n🤖 {answer}\n")

# Start chatbot
start_chat()



🏥 Medical Chatbot (Type 'exit' to quit)

💬 You: what is diabetes
🔄 Processing new query...

🤖 🧠 DIRECT: Diabetes is a group of metabolic disorders characterized by hyperglycemia, resulting from defects in insulin secretion, insulin action, or both.  This leads to elevated levels of glucose in the blood.

💬 You:  breast feeding increases the transmission of aids by how much percentage
🔄 Processing new query...

🤖 📚 RAG: Breastfeeding increases the risk of HIV transmission by 10-20%.

📎 Sources: ['docs/Medical_book.pdf']

💬 You: define diabetes
🔄 Processing new query...

🤖 🧠 DIRECT: Diabetes is a group of metabolic disorders characterized by hyperglycemia resulting from defects in insulin secretion, insulin action, or both.

💬 You: define diabetes
💡 Using cached response!

🤖 🧠 DIRECT: Diabetes is a group of metabolic disorders characterized by hyperglycemia resulting from defects in insulin secretion, insulin action, or both.

💬 You: what is the percentage of excitement reported while us

In [13]:
import gradio as gr

# Gradio function for chat
def chat_ui(user_input, history=[]):
    # Get answer from hybrid_query
    answer = cached_hybrid_query(user_input)  # NEW FUNCTION

    # Append to history
    history = history + [(user_input, answer)]
    return history, history

# Create Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## 🏥 Medical Chatbot (RAG + LLM)")
    chatbot = gr.Chatbot(label="Medical Chatbot")  # proper chat display
    user_input = gr.Textbox(
        placeholder="Ask your medical question here...",
        show_label=False
    )
    submit_btn = gr.Button("Send")

    submit_btn.click(chat_ui, inputs=[user_input, chatbot], outputs=[chatbot, chatbot])
    user_input.submit(chat_ui, inputs=[user_input, chatbot], outputs=[chatbot, chatbot])

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://96809dc68734d2102a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
